In [44]:
import vectorbt as vbt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [45]:
PLOT_BG_COLOR = "#080808"
PLOTLY_THEME = {
    "template": "plotly_dark",
    "plot_bgcolor": PLOT_BG_COLOR,
    "paper_bgcolor": PLOT_BG_COLOR,
}

pd.set_option("display.max_rows", None)

In [46]:
TICKERS = ["BTC", "ETH"]
BASE = "USD"

end_time = datetime.now()
start_time = end_time - timedelta(days=2)

pairs = [f"{t}-{BASE}" for t in TICKERS]
btc_price = vbt.YFData.download(
    pairs,
    missing_index="drop",
    start=start_time,
    end=end_time,
    interval="1m",
).get("Close")
btc_price.describe()

symbol,BTC-USD,ETH-USD
count,2740.000000,2740.000000
mean,43709.959272,2333.181211
std,1308.326161,44.989043
min,42101.953125,2261.394287
25%,42569.566406,2293.254883
50%,42852.927734,2317.426514
75%,45250.867188,2379.635742
max,45899.707031,2431.212402


## Custom indicators

In [47]:
def custom_indicator(
    close, rsi_window, ma_window, entry_threshold, exit_threshold
):
    close_5m = close.resample("5T").last()
    rsi = vbt.RSI.run(close_5m, window=rsi_window).rsi
    rsi, _ = rsi.align(
        close, broadcast_axis=0, method="ffill", join="right", axis=0
    )
    ma = vbt.MA.run(close, ma_window).ma.to_numpy()
    close = close.to_numpy()
    rsi = rsi.to_numpy()

    trend = np.where(rsi > exit_threshold, -1, 0)
    trend = np.where((rsi < entry_threshold) & (close < ma), 1, trend)

    return trend

In [48]:
ind = vbt.IndicatorFactory(
    class_name="Combination",
    short_name="comb",
    input_names=["close"],
    param_names=[
        "rsi_window",
        "ma_window",
        "entry_threshold",
        "exit_threshold",
    ],
    output_names=["trend"],
).from_apply_func(
    custom_indicator,
    rsi_window=14,
    ma_window=50,
    entry_threshold=30,
    exit_threshold=70,
    keep_pd=True,
)

In [49]:
VARIANTS = {
    "rsi_window": np.arange(10, 50, step=10, dtype=int),
    "ma_window": np.arange(20, 100, step=15, dtype=int),
    # "entry_threshold": np.arange(30, 40, step=5, dtype=int),
    # "exit_threshold": np.arange(60, 70, step=5, dtype=int)
    "entry_threshold": 30,
    "exit_threshold": 70,
}

res = ind.run(btc_price, param_product=True, **VARIANTS)
entries = res.trend == 1.0
exits = res.trend == -1.0
pf = vbt.Portfolio.from_signals(btc_price, entries, exits)
pf.stats()

/opt/conda/envs/default/lib/python3.10/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/opt/conda/envs/default/lib/python3.10/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/opt/conda/envs/default/lib/python3.10/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/opt/conda/envs/default/lib/python3.10/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set

/tmp/ipykernel_9299/2550426610.py:18: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x7feba178ecb0>. Pass column to select a single column/group.



Start                         2023-12-31 18:49:00+00:00
End                           2024-01-02 18:45:00+00:00
Period                                             2740
Start Value                                       100.0
End Value                                     99.520209
Total Return [%]                              -0.479791
Benchmark Return [%]                           4.195562
Max Gross Exposure [%]                             87.5
Total Fees Paid                                     0.0
Max Drawdown [%]                               1.935048
Max Drawdown Duration                       1380.309524
Total Trades                                   4.270833
Total Closed Trades                            3.520833
Total Open Trades                                  0.75
Open Trade PnL                                -0.619982
Win Rate [%]                                  76.756757
Best Trade [%]                                 0.597212
Worst Trade [%]                                -

In [50]:
pd.DataFrame(pf.total_return()).sort_values("total_return").tail()

total_return
comb_rsi_window comb_ma_window comb_entry_threshold comb_exit_threshold symbol               
30              95             30                   70                  BTC-USD      0.003731
                80             30                   70                  BTC-USD      0.003731
                65             30                   70                  BTC-USD      0.003731
                35             30                   70                  BTC-USD      0.003731
                20             30                   70                  BTC-USD      0.004173

In [51]:
pf.trades.records.sample(5)

,id,col,size,entry_idx,entry_price,entry_fees,exit_idx,exit_price,exit_fees,pnl,return,direction,status,parent_id
58,58,5,0.042978,1304,2311.628906,0.0,1374,2317.996338,0.0,0.273658,0.002755,0,1,58
93,93,8,0.002225,1830,44812.722656,0.0,1857,45010.730469,0.0,0.440541,0.004419,0,1,93
196,196,35,0.043853,524,2280.332520,0.0,890,2298.117676,0.0,0.779937,0.007799,0,1,196
77,77,7,0.043564,64,2295.497803,0.0,150,2291.321289,0.0,-0.181944,-0.001819,0,1,77
73,73,6,0.002216,2048,45207.351562,0.0,2081,45271.824219,0.0,0.142889,0.001426,0,1,73


In [52]:
returns = pf.total_return()
btc_returns = returns[returns.index.isin(["BTC-USD"], level="symbol")]
for pair in pairs:
    currency_returns = returns[returns.index.isin([pair], level="symbol")]
    indices = currency_returns.idxmax()
    value = currency_returns.max()
    print(f"{pair} {str(indices)}: {value}")

BTC-USD (30, 20, 30, 70, 'BTC-USD'): 0.004173438688506508
ETH-USD (40, 20, 30, 70, 'ETH-USD'): -0.001613861655403639


In [53]:
fig = returns.vbt.heatmap(
    x_level="comb_rsi_window",
    y_level="comb_ma_window",
    slider_level="symbol",
    **PLOTLY_THEME
    # theme="plotly_dark"
)
fig.show()

In [67]:
# pf.trades.close[pf.trades.close["symbol"] == "ETH-USD"]
pf.trades.close

comb_rsi_window                      10                              
comb_ma_window                       20                         35   
comb_entry_threshold                 30                         30   
comb_exit_threshold                  70                         70   
symbol                          BTC-USD      ETH-USD       BTC-USD   
Datetime                                                             
2023-12-31 18:49:00+00:00  42665.375000  2302.852051  42665.375000  \
2023-12-31 18:50:00+00:00  42649.464844  2302.156982  42649.464844   
2023-12-31 18:51:00+00:00  42647.472656  2302.685059  42647.472656   
2023-12-31 18:52:00+00:00  42655.222656  2302.334229  42655.222656   
2023-12-31 18:53:00+00:00  42638.437500  2302.064209  42638.437500   
2023-12-31 18:54:00+00:00  42635.007812  2301.564697  42635.007812   
2023-12-31 18:55:00+00:00  42636.421875  2301.704590  42636.421875   
2023-12-31 18:56:00+00:00  42637.195312  2301.707031  42637.195312   
2023-12-31 18:57:00+00:00  42626.089844  2301.643311  42626.089844   
2023-12-31 18:58:00+00:00  42636.421875  2302.368164  42636.421875   
2023-12-31 18:59:00+00:00  42638.238281  2302.459473  42638.238281   
2023-12-31 19:00:00+00:00  42642.160156  2302.697998  42642.160156   
2023-12-31 19:01:00+00:00  42645.871094  2302.635498  42645.871094   
2023-12-31 19:02:00+00:00  42634.179688  2301.849365  42634.179688   
2023-12-31 19:03:00+00:00  42638.878906  2301.867432  42638.878906   
2023-12-31 19:04:00+00:00  42640.347656  2302.147217  42640.347656   
2023-12-31 19:05:00+00:00  42650.429688  2302.937988  42650.429688   
2023-12-31 19:06:00+00:00  42643.035156  2302.333008  42643.035156   
2023-12-31 19:07:00+00:00  42646.273438  2302.269775  42646.273438   
2023-12-31 19:08:00+00:00  42623.183594  2301.304443  42623.183594   
2023-12-31 19:09:00+00:00  42600.941406  2299.994385  42600.941406   
2023-12-31 19:10:00+00:00  42626.597656  2301.230957  42626.597656   
2023-12-31 19:11:00+00:00  42635.195312  2301.075195  42635.195312   
2023-12-31 19:12:00+00:00  42628.507812  2300.791504  42628.507812   
2023-12-31 19:13:00+00:00  42626.738281  2300.267578  42626.738281   
2023-12-31 19:14:00+00:00  42622.328125  2300.215820  42622.328125   
2023-12-31 19:15:00+00:00  42623.816406  2299.505859  42623.816406   
2023-12-31 19:16:00+00:00  42619.144531  2298.492188  42619.144531   
2023-12-31 19:17:00+00:00  42617.417969  2297.262939  42617.417969   
2023-12-31 19:18:00+00:00  42608.113281  2296.970703  42608.113281   
2023-12-31 19:19:00+00:00  42601.031250  2296.387695  42601.031250   
2023-12-31 19:20:00+00:00  42599.484375  2295.785645  42599.484375   
2023-12-31 19:21:00+00:00  42604.000000  2295.679688  42604.000000   
2023-12-31 19:22:00+00:00  42605.835938  2293.593262  42605.835938   
2023-12-31 19:23:00+00:00  42586.425781  2293.178223  42586.425781   
2023-12-31 19:24:00+00:00  42601.207031  2294.014404  42601.207031   
2023-12-31 19:25:00+00:00  42607.640625  2292.808350  42607.640625   
2023-12-31 19:26:00+00:00  42616.050781  2292.690186  42616.050781   
2023-12-31 19:27:00+00:00  42631.226562  2293.219727  42631.226562   
2023-12-31 19:28:00+00:00  42640.570312  2293.477539  42640.570312   
2023-12-31 19:29:00+00:00  42649.878906  2293.818115  42649.878906   
2023-12-31 19:31:00+00:00  42647.382812  2292.805420  42647.382812   
2023-12-31 19:32:00+00:00  42647.937500  2292.969727  42647.937500   
2023-12-31 19:33:00+00:00  42637.183594  2292.778076  42637.183594   
2023-12-31 19:34:00+00:00  42599.378906  2291.328125  42599.378906   
2023-12-31 19:35:00+00:00  42597.894531  2291.626953  42597.894531   
2023-12-31 19:36:00+00:00  42596.429688  2291.506348  42596.429688   
2023-12-31 19:37:00+00:00  42607.492188  2292.066162  42607.492188   
2023-12-31 19:38:00+00:00  42614.414062  2291.933350  42614.414062   
2023-12-31 19:39:00+00:00  42611.441406  2291.845215  42611.441406   
2023-12-31 19:40:00+00:00  42623.289062  2292.394531  42623.289062   
2023-12-31